In [4]:
from thingsmri.dataset import ThingsmriLoader

# Working with ROIs

Often times, you might want to limit your analyses to specific regions of interest in the brain, either to study different anatomical locations separately from one another, or to simply save computational resources.

The THINGS-fMRI dataset comes with plenty localized ROIs. These ROIs include functionally defined regions that respond selectively to specific object categories (e.g. fusiform face area, extrastriate body part area, etc.). They also include retinotopically defined regions of interest like V1, V2, hV4, etc. Finally, the data also comes with estimated labels from the comprehensive [Glasser parcellation atlas]().

# Load the data

All this information is part of the `VoxelMetadata` which you can [download alongside the single trial fMRI responses from figshare](https://plus.figshare.com/articles/dataset/THINGS-data_fMRI_Single_Trial_Responses_table_format_/20492835/1). Let's have a look!

In [1]:
# assume this is where you've downloaded the data
!ls /Users/olivercontier/bigfri/openneuro/THINGS-data/THINGS-fMRI/derivatives/betas_csv

sub-01_ResponseData.h5      sub-02_VoxelMetadata.csv
sub-01_StimulusMetadata.csv sub-03_ResponseData.h5
sub-01_VoxelMetadata.csv    sub-03_StimulusMetadata.csv
sub-02_ResponseData.h5      sub-03_VoxelMetadata.csv
sub-02_StimulusMetadata.csv


In [5]:
# We'll use the data loader that comes with this thingsmri python module, but you may also simply load the csv files on your own as needed.
dl = ThingsmriLoader(
    thingsmri_dir='/Users/olivercontier/bigfri/openneuro/THINGS-data/THINGS-fMRI/derivatives'
)

# we're gonna chose the first subject
sub = '01'

# and load the data
responses, stimdata, voxdata = dl.load_responses(sub)

Let's have a look at the `VoxelMetadata`. As you can see, there is a lot of information included here, like noise ceiling estimates (`nc_singletrial` and `nc_testset`) or pRF parameter estimates (`prf-polarangle` or `prf-eccentricity`). But, we also have a lot of ROis encoded:

In [13]:
print(*voxdata.columns, sep='\n')

voxel_id
subject_id
voxel_x
voxel_y
voxel_z
nc_singletrial
nc_testset
splithalf_uncorrected
splithalf_corrected
prf-eccentricity
prf-polarangle
prf-rsquared
prf-size
V1
V2
V3
hV4
VO1
VO2
LO1 (prf)
LO2 (prf)
TO1
TO2
V3b
V3a
lEBA
rEBA
lFFA
rFFA
lOFA
rOFA
lSTS
rSTS
lPPA
rPPA
lRSC
rRSC
lTOS
rTOS
lLOC
rLOC
IT
glasser-V1
glasser-MST
glasser-V6
glasser-V2
glasser-V3
glasser-V4
glasser-V8
glasser-4
glasser-3b
glasser-FEF
glasser-PEF
glasser-55b
glasser-V3A
glasser-RSC
glasser-POS2
glasser-V7
glasser-IPS1
glasser-FFC
glasser-V3B
glasser-LO1
glasser-LO2
glasser-PIT
glasser-MT
glasser-A1
glasser-PSL
glasser-SFL
glasser-PCV
glasser-STV
glasser-7Pm
glasser-7m
glasser-POS1
glasser-23d
glasser-v23ab
glasser-d23ab
glasser-31pv
glasser-5m
glasser-5mv
glasser-23c
glasser-5L
glasser-24dd
glasser-24dv
glasser-7AL
glasser-SCEF
glasser-6ma
glasser-7Am
glasser-7Pl
glasser-7PC
glasser-LIPv
glasser-VIP
glasser-MIP
glasser-1
glasser-2
glasser-3a
glasser-6d
glasser-6mp
glasser-6v
glasser-p24pr
glasser-33pr
glass